# Generation: Generating a Response

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser

In [2]:
vectorstore = Chroma(persist_directory = "./intro-to-ds-lectures", 
                     embedding_function = HuggingFaceEmbeddings(model='sentence-transformers/all-MiniLM-L6-v2'))

In [3]:
len(vectorstore.get()['documents'])

42

In [4]:
retriever = vectorstore.as_retriever(search_type = 'mmr', 
                                     search_kwargs = {'k':3, 
                                                      'lambda_mult':0.7})

In [5]:
TEMPLATE = '''
Answer the following question:
{question}

To answer the question, use only the following context:
{context}

At the end of the response, specify the name of the lecture this context is taken from in the format:
Resources: *Lecture Title*
where *Lecture Title* should be substituted with the title of all resource lectures.
'''

prompt_template = PromptTemplate.from_template(TEMPLATE)

In [6]:
chat = ChatOllama(model="llama3.2",
                  num_predict=250)

In [7]:
question = "What software do data scientists use?"

In [8]:
chain = ({'context': retriever, 
         'question': RunnablePassthrough()} 
         | prompt_template 
         | chat 
         | StrOutputParser())

In [9]:
chain.invoke(question)

Update of nonexisting embedding ID: 55409552-1943-4892-949a-3b475ff9c840
Delete of nonexisting embedding ID: 55409552-1943-4892-949a-3b475ff9c840


'Based on the provided context, the software used by data scientists include:\n\n1. R\n2. Python\n3. Apache Hadoop\n4. Apache Hbase\n5. MongoDB (referenced as "Mongo DB")\n\nThese tools are mentioned in the lecture as being widely used in the field of data science.\n\nResources: Programming Languages & Software Employed in Data Science - All the Tools You Need'

In [10]:
print('Data scientists use a variety of software tools. R and Python are the two most popular tools as they can manipulate data and are integrated within multiple data and data science software platforms. They are adaptable and can solve a wide range of business and data-related problems. Hadoop is a software framework designed to handle the complexity and computational intensity of big data by distributing computational tasks on multiple computers. Additionally, Power BI, SaS, Qlik, and Tableau are top-notch examples of software designed for business intelligence visualizations.\n\nResources: Programming Languages & Software Employed in Data Science - All the Tools You Need')

Data scientists use a variety of software tools. R and Python are the two most popular tools as they can manipulate data and are integrated within multiple data and data science software platforms. They are adaptable and can solve a wide range of business and data-related problems. Hadoop is a software framework designed to handle the complexity and computational intensity of big data by distributing computational tasks on multiple computers. Additionally, Power BI, SaS, Qlik, and Tableau are top-notch examples of software designed for business intelligence visualizations.

Resources: Programming Languages & Software Employed in Data Science - All the Tools You Need
